In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd




# بناء user profile
user_profiles = df_interests.groupby("user_id")["interest"].apply(lambda x: "، ".join(x)).reset_index()

# بناء guide profile
guide_profiles = df_tags.groupby("guide_id")["tag"].apply(lambda x: "، ".join(x)).reset_index()

# دمج التوصيات مع أسماء ومناطق المستخدمين
df_cb_named_simple = df_cb_recommendations.merge(
    df_users[['id', 'name', 'region']],
    left_on="user_id",
    right_on="id",
    how="left"
).rename(columns={
    "name": "user_name",
    "region": "user_region"
}).drop(columns=["id"])

# دمج مع أسماء ومناطق المرشدين
df_cb_named_simple = df_cb_named_simple.merge(
    df_guides[['id', 'name', 'region']],
    left_on="guide_id",
    right_on="id",
    how="left"
).rename(columns={
    "name": "guide_name",
    "region": "guide_region"
}).drop(columns=["id"])

# دمج مع الاهتمامات
df_cb_named_simple = df_cb_named_simple.merge(user_profiles, on="user_id", how="left")

# دمج مع التاغز
df_cb_named_simple = df_cb_named_simple.merge(guide_profiles, on="guide_id", how="left")

# إعادة ترتيب الأعمدة + التسمية النهائية
df_cb_cleaned = df_cb_named_simple[[
    "user_name", "user_region", "interest",
    "guide_name", "guide_region", "tag", "similarity"
]].rename(columns={
    "user_name": "المستخدم",
    "user_region": "منطقة المستخدم",
    "interest": "اهتمام المستخدم",
    "guide_name": "المرشد المقترح",
    "guide_region": "منطقة المرشد",
    "tag": "توصيف المرشد",
    "similarity": "درجة التشابه"
})

# عرض النتيجة
display(df_cb_cleaned.head(10))
